In [1]:
from rdflib import Graph, Literal, RDF, URIRef,Namespace, BNode
from rdflib.namespace import FOAF , XSD,OWL,SKOS,RDFS

lmss = Namespace("http://lmss.sali.org/")
# Create a Graph
g = Graph()
# Parse in an RDF file hosted on the Internet
g.parse("SALI_new\LMSS.owl")

<Graph identifier=N7ea941fb923044d69d20b645e6b8099f (<class 'rdflib.graph.Graph'>)>

In [2]:
#Python wraper for classes in ontology
class OWLClass:
    node_label = OWL.Class.n3(namespace_manager=g.namespace_manager).replace(":","_")
    
    def __init__(self, iri, label, defination):
        self.iri = iri
        self.label = label
        self.defination = defination
        self.parents = []
        self.relations = []

class Relation:
    def __init__(self, iri, label, to_iri):
        self.iri = iri
        self.label = label
        self.to_iri = to_iri

In [3]:
#FILL CLASS DETAILS 
onto_classes = g.subjects(RDF.type, OWL.Class)
classes = []
for c in onto_classes:
    
    label = g.value(c, RDFS.label)
    if label is None:
        label = c.split("#")[-1]
        
    
    classes.append(
        OWLClass( c,label, g.value(c, SKOS.definition))
        )
    

In [4]:
#FILL RELATIONSHIPS DETAILS AND PARENTS

for clas in classes:
    
    parents = g.objects(clas.iri, RDFS.subClassOf)
    
    for p in list(parents):
        
        if type(p) == BNode:
                
            what = g.value(p, OWL.onProperty)
            where = g.value(p, OWL.someValuesFrom)

            if what is None or where is None:
                continue

    
            if what.rsplit('/', 1)[0]+'/' == str(lmss):
                label=g.value(what, RDFS.label)
            else :   
                label=what.n3(namespace_manager=g.namespace_manager)
            
            clas.relations.append(
                Relation(what,label,where)
                )
            
        elif type(p) == URIRef:
            clas.parents.append(p)
            clas.relations.append(
                Relation(
                    RDFS.subClassOf,RDFS.subClassOf.n3(namespace_manager=g.namespace_manager),p
                    )
                )
                

In [ ]:
irir_froms = []
irir_tos = []
from_texts = []
to_texts = []
Relationships = []
sentences = []


for clas in classes:

    if clas.relations == []:
        continue



    for rel in clas.relations:


        #get the label and defination of rel.to_iri
        to_iri_label = g.value(rel.to_iri, RDFS.label)

        if to_iri_label is None:
            to_iri_label = rel.to_iri.split("#")[-1]
        
        to_iri_defination = g.value(rel.to_iri, SKOS.definition)
        
        

        irir_froms.append(clas.iri)
        irir_tos.append(rel.to_iri)
        from_texts.append(clas.label)
        to_texts.append(to_iri_label)
        Relationships.append(rel.label)
        sentences.append(clas.label + " is " + rel.label + " of " + to_iri_label)

        if to_iri_defination is None or clas.defination is None:
            continue

        irir_froms.append(clas.iri)
        irir_tos.append(rel.to_iri)
        from_texts.append(clas.defination)
        to_texts.append(to_iri_defination)
        Relationships.append(rel.label)
        sentences.append(clas.defination + " is " + rel.label + " of " + to_iri_defination)


In [ ]:
import pandas as pd
# Sample data
data = {
    "iri_from": irir_froms,
    "iri_to": irir_tos,
    "from_text": from_texts,
    "to_text": to_texts,
    "relationship": Relationships,
    "sentence": sentences
}

# Create a DataFrame
df = pd.DataFrame(data)

# CSV file path
csv_file_path = 'sali_relationship_mapping.csv'

# Write the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print(f"CSV file '{csv_file_path}' created successfully.")